<a href="https://colab.research.google.com/github/Nakib-Nasrullah/Python/blob/main/test_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install wfdb numpy pandas scipy matplotlib
import wfdb
import numpy as np
import pandas as pd
import os

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.8/163.8 kB 13.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 95.4 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 2.2.2
    Uninstalling pandas-2.2.2:
      Successfully uninstalled pandas-2.2.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2.3.3 which is incompatible.


In [3]:
# Download dataset
DATA_DIR = "mitdb"
if not os.path.exists(DATA_DIR):
    wfdb.dl_database('mitdb', dl_dir=DATA_DIR)

Generating record list for: 100
Generating record list for: 101
Generating record list for: 102
Generating record list for: 103
Generating record list for: 104
Generating record list for: 105
Generating record list for: 106
Generating record list for: 107
Generating record list for: 108
Generating record list for: 109
Generating record list for: 111
Generating record list for: 112
Generating record list for: 113
Generating record list for: 114
Generating record list for: 115
Generating record list for: 116
Generating record list for: 117
Generating record list for: 118
Generating record list for: 119
Generating record list for: 121
Generating record list for: 122
Generating record list for: 123
Generating record list for: 124
Generating record list for: 200
Generating record list for: 201
Generating record list for: 202
Generating record list for: 203
Generating record list for: 205
Generating record list for: 207
Generating record list for: 208
Generating record list for: 209
Generati

In [17]:
#Parameters
WINDOW = 187
HALF = WINDOW // 2

label_map = {
    'N': 0, 'L': 0, 'R': 0, 'e': 0, 'j': 0,
    'A': 1, 'a': 1, 'J': 1, 'S': 1,
    'V': 2, 'E': 2,
    'F': 3
}

beats = []

records = sorted([f.split('.')[0] for f in os.listdir(DATA_DIR) if f.endswith('.dat')])

for record in records:
    try:
        signal, _ = wfdb.rdsamp(os.path.join(DATA_DIR, record))
        ann = wfdb.rdann(os.path.join(DATA_DIR, record), 'atr')
    except:
        continue

    ecg = signal[:, 0]
    for r, sym in zip(ann.sample, ann.symbol):
        if sym not in label_map:
            continue
        if r - HALF < 0 or r + HALF >= len(ecg):
            continue
        beat = ecg[r-HALF:r+HALF+1]
        if len(beat) == WINDOW:
            beats.append([record] + beat.tolist() + [label_map[sym]])

columns = ["record_id"] + [f"f{i}" for i in range(WINDOW)] + ["label"]
df = pd.DataFrame(beats, columns=columns)
df.to_csv("mitbih_patient_level.csv", index=False)

print(" mitbih_patient_level.csv created")

 mitbih_patient_level.csv created


In [18]:
from sklearn.model_selection import train_test_split
import pandas as pd

df = pd.read_csv("mitbih_patient_level.csv")

patients = df['record_id'].unique()

train_patients, test_patients = train_test_split(
    patients, test_size=0.30, random_state=42
)

train_df = df[df['record_id'].isin(train_patients)]
test_df  = df[df['record_id'].isin(test_patients)]

assert set(train_df['record_id']).isdisjoint(set(test_df['record_id']))

train_df.to_csv("mitbih_train_patient.csv", index=False)
test_df.to_csv("mitbih_test_patient.csv", index=False)

print(" Patient-independent split saved")

 Patient-independent split saved


In [19]:
import numpy as np
import pandas as pd

train_df = pd.read_csv("mitbih_train_patient.csv")
test_df  = pd.read_csv("mitbih_test_patient.csv")

print(train_df.shape, test_df.shape)

(69432, 189) (31994, 189)


In [20]:
X_train = train_df.iloc[:, 1:-1].values
y_train = train_df['label'].values.astype(int)

X_test = test_df.iloc[:, 1:-1].values
y_test = test_df['label'].values.astype(int)

# Per-beat normalization
X_train = (X_train - X_train.mean(axis=1, keepdims=True)) / (X_train.std(axis=1, keepdims=True) + 1e-8)
X_test  = (X_test  - X_test.mean(axis=1, keepdims=True))  / (X_test.std(axis=1, keepdims=True)  + 1e-8)

X_train = X_train.reshape(-1, 187, 1)
X_test  = X_test.reshape(-1, 187, 1)

print("CNN input:", X_train.shape)

CNN input: (69432, 187, 1)


In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Dense, Dropout, BatchNormalization, Flatten
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Conv1D(32, 5, activation='relu', input_shape=(187, 1)),
    BatchNormalization(),
    MaxPooling1D(2),

    Conv1D(64, 5, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),

    Conv1D(128, 3, activation='relu'),
    BatchNormalization(),
    MaxPooling1D(2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

model.compile(
    optimizer=Adam(0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.summary()

/usr/local/lib/python3.12/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d_3 (Conv1D)               │ (None, 183, 32)        │           192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_3           │ (None, 183, 32)        │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_3 (MaxPooling1D)  │ (None, 91, 32)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_4 (Conv1D)               │ (None, 87, 64)         │        10,304 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_4           │ (None, 87, 64)         │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_4 (MaxPooling1D)  │ (None, 43, 64)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d_5 (Conv1D)               │ (None, 41, 128)        │        24,704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_5           │ (None, 41, 128)        │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling1d_5 (MaxPooling1D)  │ (None, 20, 128)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 2560)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       327,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 4)              │           516 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 364,420 (1.39 MB)

 Trainable params: 363,972 (1.39 MB)

 Non-trainable params: 448 (1.75 KB)

In [22]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)

class_weight_dict = dict(enumerate(class_weights))
print(class_weight_dict)

{0: np.float64(0.2827404222048475), 1: np.float64(7.430650684931507), 2: np.float64(3.286255206361227), 3: np.float64(41.132701421800945)}


In [ ]:
import tensorflow as tf
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping
from tensorflow.keras.optimizers import Adam # Import Adam optimizer here

tf.config.run_functions_eagerly(True)

# Re-compile the model after setting eager execution
model.compile(
    optimizer=Adam(0.0005),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

history = model.fit(
    X_train, y_train,
    epochs=60,
    batch_size=256,
    validation_split=0.1,
    class_weight=class_weight_dict,
    callbacks=[
        ReduceLROnPlateau(patience=5),
        EarlyStopping(patience=10, restore_best_weights=True)
    ],
    verbose=1
)

/usr/local/lib/python3.12/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


Epoch 1/60
245/245 ━━━━━━━━━━━━━━━━━━━━ 86s 351ms/step - accuracy: 0.7089 - loss: 0.7540 - val_accuracy: 0.8289 - val_loss: 0.8122 - learning_rate: 5.0000e-04
Epoch 2/60
245/245 ━━━━━━━━━━━━━━━━━━━━ 85s 345ms/step - accuracy: 0.8844 - loss: 0.3001 - val_accuracy: 0.9104 - val_loss: 0.2806 - learning_rate: 5.0000e-04
Epoch 3/60
245/245 ━━━━━━━━━━━━━━━━━━━━ 140s 339ms/step - accuracy: 0.9096 - loss: 0.2103 - val_accuracy: 0.9055 - val_loss: 0.2354 - learning_rate: 5.0000e-04
Epoch 4/60
 88/245 ━━━━━━━━━━━━━━━━━━━━ 52s 334ms/step - accuracy: 0.9267 - loss: 0.1934

In [11]:
model.save("mitbih_cnn_patient_independent.h5")
print(" Model saved")

 Model saved


In [12]:
from tensorflow.keras.models import load_model

model = load_model("mitbih_cnn_patient_independent.h5")
#cell 10
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=0)
print(" Patient-independent Test Accuracy:", test_acc)

 Patient-independent Test Accuracy: 0.7087266445159912
